In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
import glob
import re
from datetime import datetime

path_jonluca = "/Users/ScottJeen/OneDrive - University of Cambridge/Research/Modelling/Emerson/jonluca_data/*.csv"
dfs = []

for fname in glob.glob(path_jonluca):
    df = pd.read_csv(fname, header=1)
    df = df.drop('Status',axis=1)
    
    # get feature name from path 
    column_name = re.findall("[A-Z][A-Z].*\.",fname)

    # rename features
    df = df.rename(columns={df.columns[0]: 'Datetime', df.columns[1]: column_name[0]})
    
    # format datatime
    datetime_format = '%b %d, %Y %H:%M:%S %p'
    df['Datetime'] = pd.to_datetime(df['Datetime'], format=datetime_format)
    df = df.set_index('Datetime')
    
    # remove multiple entries at each timestep
    df = df[~df.index.duplicated(keep='first')]

    dfs.append(df)

In [3]:
# join columns on datatime and sort alphabetically
data = dfs[0].join(dfs[1:], how='inner')
data = data.sort_index(axis=1)

# drop faulty freezer temperature sensor feature
data = data.drop(['FREEZER SLAB TEMP DOOR INSIDE.'], axis=1)

# normalize humidity features
hum = data.columns.str.contains('HUMIDITY')
data.loc[:,hum] = data.loc[:,hum] / 100

# get power data from amps (power (kW) = amps * 600V / 1000)
amp = data.columns.str.contains('COMP AMP')
data.loc[:,amp] = data.loc[:,amp] * 600 / 1000

# rename columns
new_cols = pd.Series(data.columns).str.replace('AMP', 'POWER (kW)').str.replace('.','')
new_cols = list(new_cols)
data.columns = new_cols

# add total power feature
power_features = data.columns.str.contains('POWER')
data['TOTAL POWER (kW)'] = data.loc[:,power_features].sum(axis=1)

# add energy feature (assume power is constant for 3 minute period between datapoints)
data['TOTAL ENERGY (kWh)'] = data['TOTAL POWER (kW)'] * (60/3)

In [4]:
# create datetime convertor
def datetime_conv(df, hour_format='%H', date_format='%Y-%m-%d', hour='Hour', date='Date'):
    
    # format hour feature to padded 24h 
    df[hour] = df[hour] - 1
    df[hour] = df[hour].astype(str)
    df[hour] = df[hour].str.pad(width=2, side='left', fillchar='0')

    # convert to datetime
    df[date] = pd.to_datetime(df[date], format=date_format)
    df[hour] = pd.to_datetime(df[hour], format=hour_format)

    x = []

    for index, row in df.iterrows():
        d = row.loc[date].date()
        t = row.loc[hour].time()
        x.append(dt.datetime.combine(d, t))

    df['Datetime'] = pd.Series(x)
    
    # drop old date and time cols
    df = df.drop([hour, date], axis=1)
    
    # set index to datetime
    df = df.set_index('Datetime')
    
    return df

In [5]:
import datetime as dt

# import elec data
path_elec = '/Users/ScottJeen/OneDrive - University of Cambridge/Research/Modelling/Emerson/elec_data/*.csv'

dfs_elec = []
files = [fname for fname in glob.glob(path_elec)]

# read hourly price data
hourly_price = pd.read_csv(files[0], header=3)
hourly_price = hourly_price.drop(hourly_price.columns[6:], axis=1)

# run datetime convertor
hourly_price = datetime_conv(hourly_price)

# rename columns
cols = hourly_price.columns
new_cols = {cols[0]: 'PRICE ($/MWH)',\
            cols[1]: '1 HOUR PRICE PREDICT',\
            cols[2]: '2 HOUR PRICE PREDICT',\
            cols[3]: '3 HOUR PRICE PREDICT'
           }

hourly_price = hourly_price.rename(new_cols, axis=1)

In [6]:
# read elec supply data
hourly_supply = pd.read_csv(files[1])

# run datetime convertor
hourly_supply = datetime_conv(hourly_supply, date_format='%d/%m/%Y')

# rename columns
hourly_supply = hourly_supply.rename({'Total Output': "TOTAL SUPPLY_MW",\
                                     'NUCLEAR': 'NUCLEAR_MW',\
                                      'GAS': 'GAS_MW',\
                                      'HYDRO': 'HYDRO_MW',\
                                      'WIND': 'WIND_MW',\
                                      'SOLAR': 'SOLAR_MW',\
                                      'BIOFUEL': 'BIOFUEL_MW'
                                     },\
                                     axis=1)

In [7]:
# cache timeseries index
index = data.index

# merge jonluca and prices
data = data.merge(hourly_price,\
                  left_on=[data.index.date, data.index.hour],\
                  right_on=[hourly_price.index.date, hourly_price.index.hour],\
                  how='left'
                  ).set_index(index) # keep 3 minute datetime index
                
data = data.drop(['key_0', 'key_1'], axis=1)

# merge jonluca/prices and supply
data = data.merge(hourly_supply,\
                  left_on=[data.index.date, data.index.hour],\
                  right_on=[hourly_supply.index.date, hourly_supply.index.hour],\
                  how='left'
                  ).set_index(index) # keep 3 minute datetime index

data = data.drop(['key_0', 'key_1'], axis=1)


,COOLER COMP POWER (kW) 1A,COOLER COMP POWER (kW) 1B,COOLER COMP POWER (kW) 2A,COOLER COMP POWER (kW) 2B,COOLER COMP POWER (kW) 3A,COOLER COMP POWER (kW) 3B,COOLER HUMIDITY FRONT,COOLER HUMIDITY LEFT,COOLER HUMIDITY REAR,COOLER HUMIDITY RIGHT,COOLER SLAB TEMP DOOR INSIDE LEFT,COOLER SLAB TEMP DOOR INSIDE RIGHT,COOLER SLAB TEMP DOOR OUTSIDE RIGHT,COOLER SOIL TEMP DOOR INSIDE LEFT,COOLER TEMP LEFT,COOLER TEMP RIGHT,FREEZER COMP POWER (kW) 4A,FREEZER COMP POWER (kW) 4B,FREEZER COMP POWER (kW) 5A,FREEZER COMP POWER (kW) 5B,FREEZER HUMIDITY LEFT,FREEZER HUMIDITY RIGHT,FREEZER SLAB TEMP DOOR OUTSIDE1,FREEZER SLAB TEMP DOOR OUTSIDE2,FREEZER SLAB TEMP FRONT LEFT,FREEZER SLAB TEMP REAR RIGHT,FREEZER SOIL TEMP DOOR INSIDE,FREEZER SOIL TEMP DOOR OUTSIDE1,FREEZER SOIL TEMP DOOR OUTSIDE2,FREEZER SOIL TEMP FRONT LEFT,FREEZER SOIL TEMP REAR RIGHT,FREEZER TEMP FRONT LEFT,FREEZER TEMP REAR RIGHT,GLYCOL PRESSURE RETURN,GLYCOL PRESSURE SUPPLY,GLYCOL TEMP RETURN,GLYCOL TEMP SUPPLY,TOTAL POWER (kW),TOTAL ENERGY (kWh),PRICE ($/MWH),1 HOUR PRICE PREDICT,2 HOUR PRICE PREDICT,3 HOUR PRICE PREDICT,NUCLEAR_MW,GAS_MW,HYDRO_MW,WIND_MW,SOLAR_MW,BIOFUEL_MW,TOTAL SUPPLY_MW
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-15 10:57:00,4.806,3.240,4.650,4.830,0.012,4.674,0.6998,0.6678,0.9724,0.6802,7.91,8.21,16.86,8.74,3.03,2.76,0.006,10.806,11.226,0.006,0.6232,0.6232,11.72,11.72,-18.66,-16.88,15.44,17.69,12.52,9.04,9.55,-19.61,-19.22,1.67,1.84,10.34,10.36,44.256,885.12,17.50,17.09,17.97,16.71,11414,3074,4765,603,369,28,20253
2020-07-15 11:00:00,4.686,3.246,4.596,4.818,0.012,4.650,0.7004,0.6682,0.9726,0.6792,7.89,8.25,16.79,8.70,2.99,2.72,0.012,10.158,11.364,0.012,0.6234,0.6234,11.70,11.70,-18.69,-16.78,15.41,17.69,12.50,9.04,9.51,-19.76,-19.20,1.67,1.84,10.32,10.36,43.554,871.08,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297
2020-07-15 11:03:00,4.638,3.234,4.608,4.836,0.012,4.674,0.7010,0.6690,0.9704,0.6856,7.93,8.23,16.84,8.70,3.03,2.66,0.006,11.334,0.012,0.006,0.6222,0.6222,11.70,11.70,-18.69,-16.83,15.41,17.69,12.48,9.02,9.53,-19.76,-19.25,1.67,1.84,10.30,10.34,33.360,667.20,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297
2020-07-15 11:06:00,4.776,3.228,4.650,4.758,0.012,4.686,0.7010,0.6652,0.9726,0.6930,7.95,8.23,16.86,8.74,2.95,2.66,0.012,10.932,0.012,0.006,0.6194,0.6194,11.70,11.70,-18.69,-16.81,15.44,17.69,12.48,9.02,9.53,-19.68,-19.30,1.67,1.84,10.32,10.34,33.072,661.44,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297
2020-07-15 11:09:00,4.638,3.228,4.602,4.722,0.012,4.650,0.7008,0.6644,0.9714,0.6858,7.91,8.17,16.86,8.76,2.93,2.64,0.006,10.212,0.012,0.006,0.6172,0.6172,11.67,11.67,-18.66,-16.83,15.39,17.69,12.48,9.02,9.47,-19.58,-19.25,1.67,1.84,10.30,10.32,32.088,641.76,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297


In [10]:
# create grid emission features

gas_intensity = 400 # kg/MWh

data['GRID EMISSION INTENSITY_kg/MWh'] = (data['GAS_MW'] / data['TOTAL SUPPLY_MW']) * gas_intensity
data['GRID EMISSIONS_kgs'] = data['GRID EMISSION INTENSITY_kg/MWh'] * (3/60) # 3 minute intervals

In [11]:
data

,COOLER COMP POWER (kW) 1A,COOLER COMP POWER (kW) 1B,COOLER COMP POWER (kW) 2A,COOLER COMP POWER (kW) 2B,COOLER COMP POWER (kW) 3A,COOLER COMP POWER (kW) 3B,COOLER HUMIDITY FRONT,COOLER HUMIDITY LEFT,COOLER HUMIDITY REAR,COOLER HUMIDITY RIGHT,COOLER SLAB TEMP DOOR INSIDE LEFT,COOLER SLAB TEMP DOOR INSIDE RIGHT,COOLER SLAB TEMP DOOR OUTSIDE RIGHT,COOLER SOIL TEMP DOOR INSIDE LEFT,COOLER TEMP LEFT,COOLER TEMP RIGHT,FREEZER COMP POWER (kW) 4A,FREEZER COMP POWER (kW) 4B,FREEZER COMP POWER (kW) 5A,FREEZER COMP POWER (kW) 5B,FREEZER HUMIDITY LEFT,FREEZER HUMIDITY RIGHT,FREEZER SLAB TEMP DOOR OUTSIDE1,FREEZER SLAB TEMP DOOR OUTSIDE2,FREEZER SLAB TEMP FRONT LEFT,FREEZER SLAB TEMP REAR RIGHT,FREEZER SOIL TEMP DOOR INSIDE,FREEZER SOIL TEMP DOOR OUTSIDE1,FREEZER SOIL TEMP DOOR OUTSIDE2,FREEZER SOIL TEMP FRONT LEFT,FREEZER SOIL TEMP REAR RIGHT,FREEZER TEMP FRONT LEFT,FREEZER TEMP REAR RIGHT,GLYCOL PRESSURE RETURN,GLYCOL PRESSURE SUPPLY,GLYCOL TEMP RETURN,GLYCOL TEMP SUPPLY,TOTAL POWER (kW),TOTAL ENERGY (kWh),PRICE ($/MWH),1 HOUR PRICE PREDICT,2 HOUR PRICE PREDICT,3 HOUR PRICE PREDICT,NUCLEAR_MW,GAS_MW,HYDRO_MW,WIND_MW,SOLAR_MW,BIOFUEL_MW,TOTAL SUPPLY_MW,GRID EMISSION INTENSITY_kg/MWh,GRID EMISSIONS_kgs
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-15 10:57:00,4.806,3.240,4.650,4.830,0.012,4.674,0.6998,0.6678,0.9724,0.6802,7.91,8.21,16.86,8.74,3.03,2.76,0.006,10.806,11.226,0.006,0.6232,0.6232,11.72,11.72,-18.66,-16.88,15.44,17.69,12.52,9.04,9.55,-19.61,-19.22,1.67,1.84,10.34,10.36,44.256,885.12,17.50,17.09,17.97,16.71,11414,3074,4765,603,369,28,20253,60.711993,3.035600
2020-07-15 11:00:00,4.686,3.246,4.596,4.818,0.012,4.650,0.7004,0.6682,0.9726,0.6792,7.89,8.25,16.79,8.70,2.99,2.72,0.012,10.158,11.364,0.012,0.6234,0.6234,11.70,11.70,-18.69,-16.78,15.41,17.69,12.50,9.04,9.51,-19.76,-19.20,1.67,1.84,10.32,10.36,43.554,871.08,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:03:00,4.638,3.234,4.608,4.836,0.012,4.674,0.7010,0.6690,0.9704,0.6856,7.93,8.23,16.84,8.70,3.03,2.66,0.006,11.334,0.012,0.006,0.6222,0.6222,11.70,11.70,-18.69,-16.83,15.41,17.69,12.48,9.02,9.53,-19.76,-19.25,1.67,1.84,10.30,10.34,33.360,667.20,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:06:00,4.776,3.228,4.650,4.758,0.012,4.686,0.7010,0.6652,0.9726,0.6930,7.95,8.23,16.86,8.74,2.95,2.66,0.012,10.932,0.012,0.006,0.6194,0.6194,11.70,11.70,-18.69,-16.81,15.44,17.69,12.48,9.02,9.53,-19.68,-19.30,1.67,1.84,10.32,10.34,33.072,661.44,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:09:00,4.638,3.228,4.602,4.722,0.012,4.650,0.7008,0.6644,0.9714,0.6858,7.91,8.17,16.86,8.76,2.93,2.64,0.006,10.212,0.012,0.006,0.6172,0.6172,11.67,11.67,-18.66,-16.83,15.39,17.69,12.48,9.02,9.47,-19.58,-19.25,1.67,1.84,10.30,10.32,32.088,641.76,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:12:00,4.722,3.228,4.614,4.758,0.012,4.686,0.7030,0.6642,0.9728,0.6886,7.93,8.19,16.88,8.74,2.93,2.60,0.012,10.902,0.012,0.012,0.6160,0.6160,11.67,11.67,-18.66,-16.83,15.41,17.69,12.50,9.02,9.53,-19.48,-19.27,1.67,1.84,10.30,10.32,32.958,659.16,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:15:00,4.716,3.216,4.572,4.758,0.012,4.632,0.7030,0.6628,0.9726,0.6832,7.95,8.29,16.82,8.70,2.89,2.64,0.006,11.322,9.948,0.006,0.6170,0.6170,11.65,11.65,-18.69,-16.78,15.39,17.71,12.50,9.00,9.53,-19.33,-19.22,1.67,1.84,10.30,10.30,43.188,863.76,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:18:00,4.614,3.210,4.548,4.722,0.012,4.650,0.7030,0.6618,0.9726,0.6854,7.93,8.25,16.86,8.74,2.86,2.56,0.006,10.464,10.884,0.006,0.6192,0.6192,11.65,11.65,-18.66,-16.78,15.41,17.69,12.50,9.02,9.53,-19.45,-19.22,1.67,1.84,10.28,10.32,43.116,862.32,16.81,16.75,16.79,16.80,11424,2970,4902,602,372,27,20297,58.530817,2.926541
2020-07-15 11:21:00,4.740,3.204,4.554,4.686,0.012,4.58